# Pré-processamento de dados

Vamos criar um pipeline (uma linha de processamento) que automaticamente extrai os dados de uma planilha csv, gera os corpus de treinamento e teste, gera os iteradores que vão criar os batches (lotes) e codifica as palavras de acordo com algum embedding.

Nós utilizaremos o corpus de avaliações da B2W, e estaremos interessados em apenas duas colunas: **review_text**, que sera referida daqui para frente como texto; e **overall_rating**, que sera referido apenas como nota.

Este pipeline deve conter os seguintes passos:

**filtro**: Filtrar todas as linhas cuja nota não é um valor numérico entre 0 e 5.

**partilha**: Recebe o nome do arquivo csv contendo o conjunto de dados de entrada, e recebe também o nome do diretório de saída, e as proporções dos conjuntos de dados de treinamento e de teste. A partir dos dados filtrados, altera aleatoriamente a sua ordem e gera as planilhas csv de treinamento e teste. Note que essas planilhas só devem conter as colunas de texto e de nota. As proporções sugeridas são as seguintes:

• Treinamento: 75%
• Teste: 25%

Se a quantidade de dados for pequena, pode-se aumentar a quantidade de dados de treinamento, com proporções como 85-20, ou 85-15.  ́E usual que o modelo seja salvo após o treinamento.

**codifica**: Utilizar um codificador de palavras para vetor de dados de d-dimensional. Pode ser o word2vec retreinado, mas pode ser o pacote pré-compilado do Nilc, pode ser uma rede neural do tipo Embedding, a ser treinada com os dados de treinamento.

#  Treinamento
O treinamento se inicia verificando que o número de palavras da entrada não excede um valor máximo, que  é um hiper-parâmetro do modelo. As sentenças devem ser truncadas caso a entrada exceda este valor máximo. Supondo uma ordem aleatória das sentenças do corpus de treinamento, o conjunto de sentenças devem ser organizadas em batches (lotes) de tamanho fixo, onde o tamanho de cada lote de treinamento (batch size) também é um hiper-parâmetro. Tipicamente esses valores são potencias de 2, e são limitados pelo tamanho da memória do modelo e, no caso de estar se usando uma GPU, pela quantidade de memória da GPU. Todas as sentenças no lote devem ter o mesmo tamanho, e portanto deve-se encontrar a maior sentença e completar as sentenças menores com caracteres de padding (preenchimento) <PAD>. Podemos limitar o tamanho do vocabulário em um número fixo de palavras mais frequentes, por exemplo 20.000, transformando as demais palavras em palavras desconhecidas.

O primeiro passo do treinamento consiste no **embedding das palavras** num espaço vetorial de tamanho fixo. Neste passo voce pode utilizar os valores calculados pelo programa do ep1, ou então os valores pré-calculados pelo NILC; uma terceira possibilidade é o uso de uma **camada especifica de embedding**, mas nesse exercício estamos dando preferência para o uso de algum mapeamento pré-treinado. Cada lote é submetido durante o treinamento a uma rede neural recorrente formada por elementos **LSTM ou GRU**.

Vamos experimentar dois casos. No primeiro caso utilizaremos um **Encoder unidirecional** e no segundo caso utilizaremos um **Encoder bidirecional**. Em ambos os casos, a saída do Encoder deve ser conectada a uma **rede linear densa (FeedForward)** cujo número de saídas é igual `a quantidade de classes em que a entrada pode ser classificada. Ou seja, teremos quatro experimentos variando os parâmetros de rede **uni/bi-direcional, redes LSTM e GRU**.

# Teste
Fixe um número de épocas entre 5 e 50 e treine diferentes modelos para escolher o conjunto de parâmetros que maximize a performance de cada um dos nossos 4 experimentos. Por fim, agregue em uma tabela os resultados obtidos. Uma única tabela deve reunir o resultado dos testes finais dos quatro modelos e concluir qual modelo apresenta a melhor acurácia de teste. A tabela csv deve ter o seguinte formato:

| Método               | Acurácia | Melhor (S/N) |
|----------------------|----------|--------------|
| LSTM uni-direcional  |          |              |
| LSTM bi-direcional   |          |              |
| GRU uni-direcional   |          |              |
| GRU bi-direcional    |          |              |


# Imports

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization, Embedding, Bidirectional, GRU, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical # para gerar o y como one-hot, já que é problema multiclasse
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np
np.random.seed(42)
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

%matplotlib inline

tf.__version__

'2.15.0'

In [13]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [14]:
!curl https://raw.githubusercontent.com/alan-barzilay/NLPortugues/master/Semana%2003/data/b2w-10k.csv --output 'data/b2w-10k.csv'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3919k  100 3919k    0     0  8896k      0 --:--:-- --:--:-- --:--:-- 8907k


In [15]:
usecols = ["review_text", "overall_rating"]
b2wCorpus = pd.read_csv("data/b2w-10k.csv", usecols=usecols)
b2wCorpus.head()

,overall_rating,review_text
0,4,Estou contente com a compra entrega rápida o ú...
1,4,"Por apenas R$1994.20,eu consegui comprar esse ..."
2,4,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...
3,4,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...
4,5,"A entrega foi no prazo, as americanas estão de..."


# Preprocessing

In [16]:
# check overall_rating

b2wCorpus["overall_rating"].unique()

# não tem nenhuma linha cuja nota não é um valor numérico entre 0 e 5

array([4, 5, 1, 2, 3])

In [17]:
# train, test split

random_state = 42
test_size = 0.25

x_values = b2wCorpus["review_text"]
y_values = b2wCorpus["overall_rating"]

x_train, x_test, y_train, y_test = train_test_split(x_values, y_values, random_state=random_state, test_size=test_size)

In [18]:
# embedding

import tensorflow as tf

# Exibe as GPUs disponíveis (deve exibir pelo menos uma GPU)
physical_devices = tf.config.list_physical_devices('GPU')
print("GPUs disponíveis:", physical_devices)

# Configuração para alocar memória de forma dinâmica
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Cria a camada TextVectorization
output_sequence_length = 10000
vectorizer = TextVectorization()
vectorizer.adapt(x_train) # Adaptar o vetorizador aos dados de treinamento
vocab = vectorizer.get_vocabulary() # Verificar o vocabulário criado pelo vetorizador
vectorized_train_data = vectorizer(x_train).numpy() # Vetorizar os dados de treinamento
vectorized_train_data

GPUs disponíveis: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


array([[256,   5, 300, ...,   0,   0,   0],
       [ 45,  10,   6, ...,   0,   0,   0],
       [106,  14, 107, ...,   0,   0,   0],
       ...,
       [370,  42,   2, ...,   0,   0,   0],
       [ 17,   6,   5, ...,   0,   0,   0],
       [787, 148, 764, ...,   0,   0,   0]])

In [19]:
print("Vocabulário:", vocab)

Vocabulário: ['', '[UNK]', 'o', 'e', 'a', 'de', 'produto', 'que', 'não', 'muito', 'do', 'é', 'com', 'para', 'um', 'bom', 'da', 'recomendo', 'entrega', 'no', 'em', 'uma', 'na', 'bem', 'qualidade', 'mas', 'mais', 'prazo', 'chegou', 'foi', 'excelente', 'as', 'eu', 'antes', 'se', 'comprei', 'americanas', 'como', 'meu', 'tem', 'minha', 'por', 'recebi', 'dia', 'super', 'gostei', 'até', 'compra', 'estou', 'já', 'pra', 'me', 'os', 'sem', 'boa', 'veio', 'ainda', 'ótimo', 'só', 'pois', 'mesmo', 'ser', 'loja', 'ele', 'preço', 'está', 'esse', 'aparelho', 'comprar', 'ao', 'pelo', 'celular', 'ótima', 'minhas', 'expectativas', 'nem', 'rápida', 'fácil', 'rápido', 'site', 'agora', 'tudo', 'ou', 'nada', 'ter', 'nao', 'entregue', 'dias', 'melhor', 'quando', 'Ótimo', 'uso', 'porém', 'isso', 'das', 'dentro', 'custo', 'pouco', 'são', 'pela', 'todos', 'atendeu', 'era', 'É', 'bonito', 'presente', 'tive', 'problema', 'ela', 'tempo', 'lojas', 'sempre', 'tenho', 'funciona', 'estava', 'bateria', 'atendimento', 'd

In [20]:
len(vectorized_train_data[0])

354

In [21]:
# Codificar rótulos de teste no formato one-hot
# Subtraindo 1 dos rótulos para ajustar a indexação
# A função to_categorical espera que os rótulos sejam inteiros começando de 0 até num_classes - 1.
num_classes = b2wCorpus["overall_rating"].nunique()
y_train_encoded = to_categorical(y_train - 1, num_classes=num_classes)
y_test_encoded = to_categorical(y_test - 1, num_classes=num_classes)

# Ajustar ou truncar as sequências nos dados de treinamento e teste para validação
vectorized_train_data = vectorizer(x_train).numpy()
vectorized_test_data = vectorizer(x_test).numpy()

input_len = len(vectorized_train_data[0])
vectorized_train_data_padded = pad_sequences(vectorized_train_data, maxlen=input_len, padding='post')
vectorized_test_data_padded = pad_sequences(vectorized_test_data, maxlen=input_len, padding='post')

# Models

In [22]:
%%time

# Criar o modelo
input_dim = len(vocab) + 2  # Adicionar 2 para <OOV> e <PAD>
input_len = len(vectorized_train_data[0])
embedding_dim = 50
output_dim = 10
num_classes = b2wCorpus["overall_rating"].nunique()

optimizer = 'adam'
loss = 'categorical_crossentropy'
epochs = 20
metrics = ['AUC']
epochs=30
batch_size=50

CPU times: user 896 µs, sys: 0 ns, total: 896 µs
Wall time: 762 µs


In [23]:
%%time

lstm_uni_model = Sequential()
lstm_uni_model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_len))

lstm_uni_model.add(LSTM(64, return_sequences=True))  # Use return_sequences=True para conectar camadas LSTM em sequência
lstm_uni_model.add(Dropout(0.5))
lstm_uni_model.add(LSTM(64, return_sequences=True))
lstm_uni_model.add(Dropout(0.5))
lstm_uni_model.add(LSTM(64))
lstm_uni_model.add(Dropout(0.5))

lstm_uni_model.add(Dense(64, activation='relu', input_dim=input_dim))
lstm_uni_model.add(Dense(num_classes, activation='softmax'))  # Camada de saída com softmax para problemas multiclasse
lstm_uni_model.compile(optimizer=optimizer, loss=loss, metrics=metrics) # Compilar o modelo
lstm_uni_model.fit(vectorized_train_data_padded, y_train_encoded, epochs=epochs, batch_size=batch_size) # treina

Epoch 1/30
150/150 [==============================] - 35s 172ms/step - loss: 1.4840 - auc: 0.6775
Epoch 2/30
150/150 [==============================] - 17s 113ms/step - loss: 1.4710 - auc: 0.6848
Epoch 3/30
150/150 [==============================] - 11s 73ms/step - loss: 1.4693 - auc: 0.6856
Epoch 4/30
150/150 [==============================] - 9s 59ms/step - loss: 1.4684 - auc: 0.6850
Epoch 5/30
150/150 [==============================] - 8s 56ms/step - loss: 1.4686 - auc: 0.6848
Epoch 6/30
150/150 [==============================] - 7s 48ms/step - loss: 1.4673 - auc: 0.6849
Epoch 7/30
150/150 [==============================] - 8s 51ms/step - loss: 1.4659 - auc: 0.6858
Epoch 8/30
150/150 [==============================] - 7s 49ms/step - loss: 1.4665 - auc: 0.6861
Epoch 9/30
150/150 [==============================] - 6s 40ms/step - loss: 1.4661 - auc: 0.6854
Epoch 10/30
150/150 [==============================] - 6s 39ms/step - loss: 1.4663 - auc: 0.6844
Epoch 11/30
150/150 [=============

In [24]:
%%time

gru_uni_model = Sequential()
gru_uni_model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_len))

gru_uni_model.add(GRU(64, return_sequences=True))
gru_uni_model.add(Dropout(0.5))
gru_uni_model.add(GRU(64, return_sequences=True))
gru_uni_model.add(Dropout(0.5))
gru_uni_model.add(GRU(64))
gru_uni_model.add(Dropout(0.5))

gru_uni_model.add(Dense(64, activation='relu'))
gru_uni_model.add(Dense(num_classes, activation='softmax')) # camada de saída
gru_uni_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
gru_uni_model.fit(vectorized_train_data_padded, y_train_encoded, epochs=epochs, batch_size=batch_size)

Epoch 1/30
150/150 [==============================] - 24s 114ms/step - loss: 1.4821 - auc: 0.6783
Epoch 2/30
150/150 [==============================] - 12s 82ms/step - loss: 1.4691 - auc: 0.6850
Epoch 3/30
150/150 [==============================] - 10s 67ms/step - loss: 1.4700 - auc: 0.6852
Epoch 4/30
150/150 [==============================] - 7s 46ms/step - loss: 1.4692 - auc: 0.6839
Epoch 5/30
150/150 [==============================] - 8s 57ms/step - loss: 1.4680 - auc: 0.6849
Epoch 6/30
150/150 [==============================] - 7s 44ms/step - loss: 1.4661 - auc: 0.6860
Epoch 7/30
150/150 [==============================] - 7s 48ms/step - loss: 1.4658 - auc: 0.6854
Epoch 8/30
150/150 [==============================] - 6s 39ms/step - loss: 1.4663 - auc: 0.6843
Epoch 9/30
150/150 [==============================] - 7s 48ms/step - loss: 1.4662 - auc: 0.6838
Epoch 10/30
150/150 [==============================] - 6s 38ms/step - loss: 1.4662 - auc: 0.6848
Epoch 11/30
150/150 [==============

In [25]:
%%time

lstm_bi_model = Sequential()
lstm_bi_model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_len))

lstm_bi_model.add(Bidirectional(LSTM(64, return_sequences=True)))
lstm_bi_model.add(Dropout(0.5))
lstm_bi_model.add(Bidirectional(LSTM(64, return_sequences=True)))
lstm_bi_model.add(Dropout(0.5))
lstm_bi_model.add(Bidirectional(LSTM(64)))
lstm_bi_model.add(Dropout(0.5))

lstm_bi_model.add(Dense(64, activation='relu'))
lstm_bi_model.add(Dense(num_classes, activation='softmax')) # camada de saída
lstm_bi_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
lstm_bi_model.fit(vectorized_train_data_padded, y_train_encoded, epochs=5, batch_size=batch_size)

Epoch 1/5
150/150 [==============================] - 33s 148ms/step - loss: 1.2984 - auc: 0.7712
Epoch 2/5
150/150 [==============================] - 17s 115ms/step - loss: 1.0277 - auc: 0.8616
Epoch 3/5
150/150 [==============================] - 14s 94ms/step - loss: 0.9168 - auc: 0.8883
Epoch 4/5
150/150 [==============================] - 13s 88ms/step - loss: 0.8380 - auc: 0.9069
Epoch 5/5
150/150 [==============================] - 12s 83ms/step - loss: 0.7514 - auc: 0.9249
CPU times: user 1min 34s, sys: 2.57 s, total: 1min 36s
Wall time: 1min 31s


In [26]:
%%time

gru_bi_model = Sequential()
gru_bi_model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_len))

gru_bi_model.add(Bidirectional(GRU(64, return_sequences=True)))
gru_bi_model.add(Dropout(0.5))
gru_bi_model.add(Bidirectional(GRU(64, return_sequences=True)))
gru_bi_model.add(Dropout(0.5))
gru_bi_model.add(Bidirectional(GRU(64)))
gru_bi_model.add(Dropout(0.5))

gru_bi_model.add(Dense(64, activation='relu')) # camada dense
gru_bi_model.add(Dense(num_classes, activation='softmax')) # camada de saída

gru_bi_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
gru_bi_model.fit(vectorized_train_data_padded, y_train_encoded, epochs=5, batch_size=batch_size)

Epoch 1/5
150/150 [==============================] - 34s 159ms/step - loss: 1.3440 - auc: 0.7498
Epoch 2/5
150/150 [==============================] - 16s 108ms/step - loss: 1.0416 - auc: 0.8582
Epoch 3/5
150/150 [==============================] - 14s 96ms/step - loss: 0.8864 - auc: 0.8966
Epoch 4/5
150/150 [==============================] - 12s 83ms/step - loss: 0.7837 - auc: 0.9190
Epoch 5/5
150/150 [==============================] - 12s 80ms/step - loss: 0.6748 - auc: 0.9402
CPU times: user 1min 32s, sys: 2.39 s, total: 1min 35s
Wall time: 1min 31s


# Summaries

In [27]:
lstm_uni_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 354, 10)           127580    
                                                                 
 lstm (LSTM)                 (None, 354, 64)           19200     
                                                                 
 dropout (Dropout)           (None, 354, 64)           0         
                                                                 
 lstm_1 (LSTM)               (None, 354, 64)           33024     
                                                                 
 dropout_1 (Dropout)         (None, 354, 64)           0         
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0

In [28]:
lstm_bi_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 354, 10)           127580    
                                                                 
 bidirectional (Bidirection  (None, 354, 128)          38400     
 al)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 354, 128)          0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 354, 128)          98816     
 onal)                                                           
                                                                 
 dropout_7 (Dropout)         (None, 354, 128)          0         
                                                                 
 bidirectional_2 (Bidirecti  (None, 128)              

In [29]:
gru_uni_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 354, 10)           127580    
                                                                 
 gru (GRU)                   (None, 354, 64)           14592     
                                                                 
 dropout_3 (Dropout)         (None, 354, 64)           0         
                                                                 
 gru_1 (GRU)                 (None, 354, 64)           24960     
                                                                 
 dropout_4 (Dropout)         (None, 354, 64)           0         
                                                                 
 gru_2 (GRU)                 (None, 64)                24960     
                                                                 
 dropout_5 (Dropout)         (None, 64)               

In [30]:
gru_bi_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 354, 10)           127580    
                                                                 
 bidirectional_3 (Bidirecti  (None, 354, 128)          29184     
 onal)                                                           
                                                                 
 dropout_9 (Dropout)         (None, 354, 128)          0         
                                                                 
 bidirectional_4 (Bidirecti  (None, 354, 128)          74496     
 onal)                                                           
                                                                 
 dropout_10 (Dropout)        (None, 354, 128)          0         
                                                                 
 bidirectional_5 (Bidirecti  (None, 128)              

# Validation

In [33]:
lstm_uni_model_loss, lstm_uni_model_accuracy = lstm_uni_model.evaluate(vectorized_test_data_padded, y_test_encoded)
lstm_bi_model_loss, lstm_bi_model_accuracy = lstm_bi_model.evaluate(vectorized_test_data_padded, y_test_encoded)
gru_uni_model_loss, gru_uni_model_accuracy = gru_uni_model.evaluate(vectorized_test_data_padded, y_test_encoded)
gru_bi_model_loss, gru_bi_model_accuracy = gru_bi_model.evaluate(vectorized_test_data_padded, y_test_encoded)

print(f"Perda e Acurácia do modelo LSTM unidirecional nos dados de teste: {lstm_uni_model_loss}, {lstm_uni_model_accuracy}")
print(f"Perda e Acurácia do modelo LSTM bidirecional nos dados de teste: {lstm_bi_model_loss}, {lstm_bi_model_accuracy}")
print(f"Perda e Acurácia do modelo GRU unidirecional nos dados de teste: {gru_uni_model_loss}, {gru_uni_model_accuracy}")
print(f"Perda e Acurácia do modelo GRU bidirecional nos dados de teste: {gru_bi_model_loss}, {gru_bi_model_accuracy}")

79/79 [==============================] - 3s 36ms/step - loss: 1.2331 - auc: 0.8401
Perda e Acurácia do modelo LSTM unidirecional nos dados de teste: 1.4481182098388672, 0.6972000002861023
Perda e Acurácia do modelo LSTM bidirecional nos dados de teste: 1.172155499458313, 0.8390766978263855
Perda e Acurácia do modelo GRU unidirecional nos dados de teste: 1.4480605125427246, 0.6972000002861023
Perda e Acurácia do modelo GRU bidirecional nos dados de teste: 1.2331181764602661, 0.8401130437850952


In [42]:
# agrupando resultados no df
dict_results = {}
dict_results["Método"] = ["LSTM Unidirecional", "LSTM Bidirecional", "GRU Unidirecional", "GRU Bidirecional"]
dict_results["Acurácia"] = [lstm_uni_model_accuracy, lstm_bi_model_accuracy, gru_uni_model_accuracy, gru_bi_model_accuracy]

df_results = pd.DataFrame(dict_results)

# Adicionar a coluna "Melhor (S/N)" com base na maior acurácia
df_results["Melhor (S/N)"] = ["S" if acc == df_results["Acurácia"].max() else "N" for acc in df_results["Acurácia"]]
df_results = df_results.sort_values(by="Acurácia", ascending=False)
df_results

,Método,Acurácia,Melhor (S/N)
3,GRU Bidirecional,0.840113,S
1,LSTM Bidirecional,0.839077,N
0,LSTM Unidirecional,0.697200,N
2,GRU Unidirecional,0.697200,N


In [43]:
df_results.to_csv("./data/df_results.csv")